In [1]:
# Import dependencies
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LinearRegression

# **************************************
# keras.models.Sequential()
# **************************************
# import tensorflow as tf
# from tensorflow.kera.callbacks import ModelCheckpoint


In [2]:
# Import and read data
# combined_df = getfromdatabase{postres}

from utils_common import get_db_session
db_session = get_db_session()
combined_df = pd.read_sql_table('<tablename>', db_session)

NotImplementedError: read_sql_table only supported for SQLAlchemy connectable.

In [ ]:
# Data is explored and scrubbed elsewhere.

In [3]:
from utils_common import encode_dataframe

In [ ]:
combine_df = encode_dataframe(combined_df)
combine_df.head()

In [4]:
# **************************************
# Configuration Variables
# **************************************

# ******************
# First Layer Configuration
kS_nn_first_units       = 8
kS_nn_first_activation  = "relu"

# ******************
# Output Layer Configuration
kS_nn_output_units      = 1
kS_nn_output_activation = "sigmoid"

# ******************
# Deep Learning Configuration
kS_nn_deep_layers       = [
    # Comment out the next two lines to disable "deep learning"
    {"units": 16,
     "activation": "relu"},

    # Add more layers with
    #    {"units": <units>,
    #     "activation": <activation>},
                     ]

# ******************
# Compiler Configuration
kS_nn_compile_loss      = "binary_crossentropy"
kS_nn_compile_optimizer = "adam"
kS_nn_compile_metrics   = ["accuracy"]


# ******************
# Training Configuration
kS_nn_train_epochs      = 100

# ******************
# Output File
kS_nn_file              = "kS-Enlistment_Prediction.h5"